In [1]:
import os
import numpy as np
import numpy as np
# import torch
# from arsf_envi_reader import envi_header
import shutil
import os
import json
import math
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from osgeo import gdal,ogr,osr
from scipy.optimize import curve_fit
# from tqdm import tqdm
# import multiprocess as mp
from scipy import ndimage
from numpy import trapz
from scipy import stats
import rasterio

In [2]:
npy_folder = r'E:\wenqu\numpy_file\npy_file\site1b'  # where .npy files are stored
reference_tif = r'D:\wenqu\chapter1_2\updated_uas_modeling\uas_trait_map\site1b_ldmc.tif'    # an existing .tif file to copy georeferencing info


In [3]:
ref_ds = gdal.Open(reference_tif)
width = ref_ds.RasterXSize
height = ref_ds.RasterYSize
print(width, height)

10637 19347


# pn

In [19]:
output_tif = r'D:\wenqu\chapter1_2\updated_uas_modeling\uas_trait_map\site1b_pn.tif'


# Your list of selected bands
band_names = ['b26_std', 'b53_mean', 'b78_std', 'b79_std', 'b46_mean',
       'b36_mean', 'b95_std', 'b48_std', 'b80_std', 'b103_std',
       'b102_std', 'b27_mean', 'b56_std', 'b10_std', 'b116_mean',
       'b111_mean', 'b8_std', 'b92_mean', 'b117_std', 'b74_std',
       'b81_std', 'b100_std', 'b25_std', 'b99_std', 'b112_mean',
       'b101_std', 'b6_std', 'b14_mean', 'b44_std', 'b119_mean',
       'b106_std', 'b28_mean', 'b48_mean', 'b121_mean', 'b9_mean',
       'b64_mean', 'b21_mean', 'b54_mean', 'b34_mean', 'b72_mean',
       'b88_std', 'b113_mean', 'b26_mean', 'b64_std', 'b16_mean',
       'b119_std', 'b74_mean', 'b96_std', 'b98_std', 'b13_mean',
       'b111_std', 'b100_mean', 'b4_mean', 'b29_mean', 'b1_mean',
       'b121_std', 'b68_mean', 'b118_std', 'b104_mean', 'b84_std',
       'b25_mean', 'b93_std', 'b60_mean', 'b122_mean', 'b18_mean',
       'b58_mean', 'b104_std', 'b113_std', 'b118_mean']


plsr_coefficients  = np.array([-6.251628768478961,
 32.16297703050638,
 -15.068628308076393,
 5.428252214364907,
 69.84643119591176,
 -27.685949618955107,
 4.40287919999009,
 5.735672607996701,
 34.02960449465758,
 -0.6118783449168849,
 16.023021175982127,
 -1.571890294710857,
 5.267401538413659,
 -4.667570565393137,
 47.49298377747452,
 40.13808656632717,
 14.338359073283067,
 -112.51008777647093,
 -1.8522656999577334,
 -14.03968939176024,
 -4.501391016442681,
 5.772067445321137,
 3.663699989711924,
 -3.6502597458464097,
 -12.996227746640129,
 3.4235185386475186,
 49.97093653774433,
 12.152042115741725,
 7.282388953435546,
 -7.946552505788617,
 -7.17802053622849,
 -15.041178522050824,
 116.62806221639276,
 -22.571467877855625,
 83.11794281129177,
 44.0430480907075,
 6.42955385330087,
 23.08306612593665,
 -75.66751432804995,
 47.65465880879377,
 -6.322995188871767,
 29.805238789786884,
 6.762371277451969,
 -6.2953441369896685,
 -126.35524998261204,
 -6.979214088893311,
 15.539067359637967,
 2.161169527958224,
 19.9405480781016,
 45.43002741940881,
 4.548550013410324,
 80.54735916258583,
 -40.626184646411936,
 -90.23912829443297,
 -18.55331199579962,
 -0.26117975029990365,
 -77.25082719411982,
 -1.105566224276406,
 135.5043067044733,
 -28.821316470104723,
 -57.26555274490973,
 6.16437715297528,
 -215.08454814638515,
 -88.60460706621623,
 132.06259297067027,
 141.68558802664958,
 -16.38492442286299,
 -8.091528190954755,
 -141.63217513843347])

intercept = 0.5205581
trait_map = None


# Initialize trait map
trait_map = None
first_band = True

# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")


Successfully created trait map: D:\wenqu\chapter1_2\updated_uas_modeling\uas_trait_map\site1b_pn.tif
Trait map stats - Min: -53.37, Max: 5.79, Mean: -4.94


# pc

# scale

In [4]:
output_tif = r'D:\wenqu\chapter1_2\updated_uas_modeling\uas_trait_map\projection\site6_pc.tif'


# Your list of selected bands
band_names = ['b59_mean', 'b16_mean', 'b104_mean', 'b54_mean', 'b95_std',
       'b26_std', 'b108_std', 'b32_mean', 'b96_mean', 'b48_std',
       'b58_std', 'b64_std', 'b110_std', 'b113_mean', 'b82_mean',
       'b40_mean', 'b48_mean', 'b12_mean', 'b46_mean', 'b42_mean',
       'b99_std', 'b60_mean', 'b102_std', 'b103_std', 'b25_std',
       'b22_std', 'b91_mean', 'b62_std', 'b70_mean', 'b13_mean',
       'b29_mean', 'b86_std', 'b4_mean', 'b72_std', 'b88_std', 'b4_std',
       'b92_std', 'b104_std', 'b113_std', 'b84_mean', 'b56_std',
       'b115_std', 'b122_mean', 'b62_mean', 'b93_std', 'b109_std',
       'b80_std', 'b100_std', 'b22_mean', 'b9_mean', 'b120_mean',
       'b66_std', 'b112_std']


plsr_coefficients  = np.array([9.757854058979868,
 -7.622386914650275,
 29.46640540308122,
 15.307956670681252,
 0.16453483873701957,
 5.7544409205014935,
 -0.8148820969585087,
 50.959328247295616,
 16.06198321119629,
 8.310811320281102,
 5.496855297407003,
 -2.438807321269299,
 3.967178065745288,
 5.560741368700168,
 -36.476238854335705,
 -46.64922548406604,
 -15.674191914325846,
 27.48603089065987,
 76.6858637109823,
 -6.789668573615507,
 2.5390663068372046,
 16.494174647914164,
 4.194203436402344,
 -1.2261055302580437,
 0.20111499327527654,
 -17.17474816569315,
 -5.30090233013682,
 4.530475774095848,
 -14.149015805120841,
 -1.146937014665361,
 -11.088280514529526,
 -1.0606915953440665,
 -10.402628547525275,
 -1.0917836626285995,
 -4.766451176721566,
 12.916891045838645,
 2.0016207836291375,
 4.300798187332688,
 -1.1238889510459504,
 3.8302757536641447,
 14.497806324667247,
 -2.439503742392824,
 -16.440812301256287,
 -18.773732245680023,
 -2.4515791159011573,
 3.59730927177485,
 -2.419259157594773,
 5.316330149457974,
 -58.78132128289836,
 12.273047610033084,
 -22.184823049390566,
 -16.359973888873753,
 -10.547654506156714])

intercept = 3.69572846
trait_map = None


# Initialize trait map
trait_map = None
first_band = True

# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")


Successfully created trait map: D:\wenqu\chapter1_2\updated_uas_modeling\uas_trait_map\projection\site6_pc.tif
Trait map stats - Min: -12.14, Max: 4.96, Mean: -0.11


# LA

# scale

In [4]:
output_tif = r'D:\wenqu\chapter1_2\updated_uas_modeling\uas_trait_map\site6_la.tif'


# # Your list of selected bands
band_names = ['b19_mean', 'b94_std', 'b27_std', 'b8_mean', 'b50_std', 'b29_mean',
       'b78_std', 'b30_std', 'b44_std', 'b14_mean', 'b68_mean',
       'b39_mean', 'b86_mean', 'b113_mean', 'b62_mean', 'b119_std',
       'b45_std', 'b79_std', 'b36_mean', 'b52_mean', 'b7_mean',
       'b53_mean', 'b94_mean', 'b72_mean', 'b92_mean', 'b44_mean',
       'b84_mean', 'b112_mean', 'b64_std', 'b91_std', 'b46_mean',
       'b107_std', 'b74_std', 'b10_std', 'b89_std', 'b5_std', 'b8_std',
       'b88_std', 'b122_mean', 'b34_mean', 'b55_mean', 'b50_mean',
       'b2_mean', 'b78_mean', 'b5_mean', 'b113_std', 'b121_std',
       'b1_mean', 'b106_std', 'b104_mean', 'b23_mean', 'b6_std',
       'b30_mean', 'b105_std', 'b114_mean', 'b74_mean', 'b103_std',
       'b32_mean', 'b16_mean', 'b100_std', 'b9_mean', 'b84_std',
       'b3_mean', 'b64_mean', 'b93_std', 'b118_std', 'b20_mean',
       'b92_std', 'b24_mean', 'b17_mean', 'b22_mean']

plsr_coefficients  = np.array([73.43332945115424,
 1.8997071890983306,
 -4.491064038156993,
 -94.3073208396375,
 11.370152240599866,
 6.684057739210883,
 -17.37233750941516,
 -36.55411910672085,
 25.83174613599107,
 -157.33146854618656,
 -135.19597313847248,
 77.12165081708605,
 59.99373576156026,
 -40.443119156803256,
 -168.66580745203618,
 -7.901684152532362,
 7.503367813090542,
 5.578642587316881,
 240.0377767255638,
 211.68909284247232,
 51.08812869986964,
 -84.0047454176666,
 -185.52209469637157,
 109.44052379855532,
 -246.58350519974852,
 -95.6942755183521,
 -48.89093692706882,
 -113.82539343467575,
 39.92044951290615,
 7.558960927578383,
 -44.444525648127474,
 1.1626740772228747,
 -20.94643182989812,
 35.29212008688127,
 -1.3465506803452922,
 26.63026088080052,
 36.231940090125875,
 -43.81384109253915,
 -200.98510258832567,
 -278.4289688211392,
 -15.190023760466007,
 259.4251606170992,
 25.222293740004723,
 162.7808405426492,
 -108.19812140906629,
 -6.482932290214733,
 -5.081460661941372,
 -41.09827756378587,
 -27.657053261406745,
 202.83520835189552,
 -202.1457461594415,
 77.59105338474421,
 -295.7492962524598,
 -5.27860912702778,
 189.97739626104556,
 133.40543332855518,
 0.4788474645621897,
 474.5247843511139,
 -421.75304602435887,
 -54.302699452906616,
 193.56447095091298,
 -24.417216569762758,
 -70.99339520800018,
 -106.08095937136737,
 10.395075093905708,
 41.753602637985125,
 529.2302004567458,
 45.619472495204135,
 523.7651806092473,
 268.08222925064274,
 -770.0902580949859])


intercept = 3.10610339
trait_map = None
# Loop through bands and apply coefficients
for band_name, coef in zip(band_names, plsr_coefficients):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    if not os.path.exists(band_path):
        print(f"Missing file: {band_path}")
        continue
    band_array = np.load(band_path)

    if trait_map is None:
        trait_map = np.zeros_like(band_array)

    trait_map += band_array * coef

# Add intercept if available
trait_map += intercept

In [5]:
# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")

Successfully created trait map: D:\wenqu\chapter1_2\updated_uas_modeling\uas_trait_map\site6_la.tif
Trait map stats - Min: -81.83, Max: 11.46, Mean: -3.17


# SLA

# scale

In [44]:
output_tif = r'D:\wenqu\chapter1_2\uas_trait_map\site2c_sla.tif'


# Your list of selected bands
band_names = ['b63_mean', 'b60_std', 'b12_mean', 'b122_std', 'b12_std',
       'b48_std', 'b116_mean', 'b44_std', 'b19_mean', 'b108_mean',
       'b104_mean', 'b9_mean', 'b107_std', 'b100_mean', 'b43_std',
       'b1_mean', 'b61_std', 'b14_mean', 'b99_mean', 'b111_std',
       'b116_std', 'b105_std', 'b121_mean', 'b29_std', 'b64_std',
       'b106_mean', 'b74_mean', 'b8_std', 'b113_std', 'b2_mean',
       'b102_std', 'b23_std', 'b108_std', 'b24_std', 'b7_mean', 'b14_std',
       'b4_mean', 'b103_std', 'b6_std', 'b18_std', 'b8_mean', 'b17_std',
       'b117_std']

plsr_coefficients  = np.array([-18.780518589012903,
 21.812587972066375,
 59.24906429829236,
 -16.486924518027195,
 -177.79598538643341,
 21.475685250626956,
 -5.266684357719652,
 -49.85713281804315,
 -23.240284388367083,
 -7.414619313755988,
 6.696539728824969,
 104.60605912036955,
 -1.9899411614893365,
 7.512265637462434,
 -80.00838809638864,
 -8.007488211112067,
 47.601240734437525,
 -11.640822731956085,
 7.469762120577528,
 -9.697986942935145,
 3.4581356691881835,
 2.2614584164766773,
 -4.06820863718341,
 59.48690740719419,
 25.4784774777752,
 7.627451580273314,
 -9.913642440336956,
 -171.57863893107503,
 18.092340239555114,
 36.48083556166394,
 -17.37963127547037,
 -152.08530788595624,
 25.986405540996824,
 -121.48929023190499,
 108.15415899365468,
 167.40728540916396,
 -107.64765518542235,
 -36.82596239329592,
 203.0008252970337,
 226.0128983347566,
 -193.39820622140186,
 89.14600581936627,
 28.62352476055483])

intercept = 3.36165483
trait_map = None

# Loop through bands and apply coefficients
for band_name, coef in zip(band_names, plsr_coefficients):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    if not os.path.exists(band_path):
        print(f"Missing file: {band_path}")
        continue
    band_array = np.load(band_path)

    if trait_map is None:
        trait_map = np.zeros_like(band_array)

    trait_map += band_array * coef

# Add intercept if available
trait_map += intercept


In [45]:
# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")

Successfully created trait map: D:\wenqu\chapter1_2\uas_trait_map\site2c_sla.tif
Trait map stats - Min: -1.32, Max: 9.84, Mean: 1.93


# LDMC

# scale

In [46]:
output_tif = r'D:\wenqu\chapter1_2\uas_trait_map\site2c_ldmc.tif'


# Your list of selected bands
band_names = ['b28_std', 'b121_mean', 'b118_std', 'b101_mean', 'b120_mean',
       'b100_mean', 'b52_mean', 'b110_std', 'b97_std', 'b9_std',
       'b62_mean', 'b54_std', 'b116_std', 'b59_mean', 'b118_mean',
       'b73_std', 'b23_mean', 'b25_mean', 'b61_mean', 'b10_std',
       'b29_mean', 'b107_mean', 'b83_std', 'b6_std', 'b53_mean',
       'b42_std', 'b105_std', 'b115_mean', 'b24_std', 'b11_mean',
       'b104_std', 'b31_std', 'b114_std', 'b37_std', 'b9_mean', 'b16_std',
       'b47_std', 'b25_std', 'b122_mean', 'b35_std', 'b11_std', 'b4_mean',
       'b6_mean', 'b14_std', 'b117_mean', 'b1_std', 'b12_std', 'b94_std',
       'b5_std', 'b103_std', 'b3_mean', 'b19_mean', 'b8_std', 'b4_std',
       'b107_std', 'b29_std', 'b1_mean', 'b112_mean', 'b112_std']

plsr_coefficients  = np.array([249.16447700670548,
 4.551407603216511,
 -4.495397128285825,
 -1.3506656173162146,
 5.882207820036758,
 -0.5002515626741606,
 21.70333149311073,
 -15.729033697080295,
 -10.8730573727861,
 -170.5431597445616,
 -3.3695234854277696,
 -31.425883367807483,
 -16.64418637784629,
 -53.37513180891204,
 -4.078993928266054,
 -2.0579935966122394,
 -95.49116004644645,
 4.560360855230688,
 -21.23084572669234,
 -87.41339656876097,
 64.92935372643433,
 -17.830978533336616,
 22.996046788561035,
 247.9405273668483,
 81.41035560250417,
 -191.36132016569013,
 -20.014400006905653,
 -7.458728274604349,
 12.651701216407869,
 -87.05118594895038,
 -6.4171354690177225,
 -84.70700934713639,
 -22.167001950823,
 -92.67637091268219,
 83.70443812545916,
 62.473180666707826,
 198.96391001813856,
 50.02247845320401,
 8.76451315707005,
 143.52764300151932,
 -200.13405608302875,
 -147.21878636705634,
 -86.79250659311046,
 -302.8517550697865,
 0.008012940754274838,
 -33.844143010149764,
 183.5210834583639,
 -46.758252661735284,
 207.46333453672545,
 43.695680388734466,
 67.64534756267287,
 72.66489342752969,
 367.43173661420406,
 -207.55747042602871,
 41.847805068726814,
 -181.01968202389716,
 54.53960791823286,
 10.49885195940063,
 39.836174519769635])

intercept =-0.61511644
trait_map = None

# Loop through bands and apply coefficients
for band_name, coef in zip(band_names, plsr_coefficients):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    if not os.path.exists(band_path):
        print(f"Missing file: {band_path}")
        continue
    band_array = np.load(band_path)

    if trait_map is None:
        trait_map = np.zeros_like(band_array)

    trait_map += band_array * coef

# Add intercept if available
trait_map += intercept

# Initialize trait map
trait_map = None
first_band = True

# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")

Successfully created trait map: D:\wenqu\chapter1_2\uas_trait_map\site2c_ldmc.tif
Trait map stats - Min: -6.07, Max: 4.59, Mean: -0.19


# d13c

# scale

In [4]:
output_tif = r'D:\wenqu\chapter1_2\uas_trait_map\site6_d13c.tif'


# Your list of selected bands
band_names = ['b82_mean', 'b56_mean', 'b75_mean', 'b67_mean', 'b99_mean',
       'b97_mean', 'b57_mean', 'b35_mean', 'b26_mean', 'b5_mean',
       'b116_mean', 'b17_mean', 'b1_mean', 'b14_mean', 'b73_mean',
       'b28_mean', 'b3_mean', 'b39_mean', 'b51_mean', 'b122_mean',
       'b40_mean', 'b30_mean', 'b49_mean', 'b42_mean', 'b76_mean',
       'b48_mean', 'b8_mean', 'b34_mean', 'b121_mean', 'b52_mean',
       'b15_mean', 'b7_mean']

plsr_coefficients  = np.array([236.61852623486863,
 -482.88222925347776,
 138.77216745347565,
 -44.673199569661506,
 -130.83854381839146,
 -69.32322347897814,
 -130.99011762271542,
 174.92483319157319,
 86.31135633974849,
 103.09103836913387,
 44.41173006786578,
 -0.13247107610119974,
 83.19520249834109,
 525.2716626984129,
 78.22438490255212,
 290.4246751239607,
 -97.49652904815359,
 -232.4310991099464,
 161.20342046086108,
 104.632806150325,
 -387.46969869396503,
 116.29419474030463,
 138.606982935393,
 -116.52149104887903,
 -357.40088662930526,
 -128.67990761302707,
 -366.92702046262247,
 -364.4010164924634,
 -70.19222226778966,
 765.1253660579733,
 458.31738032251434,
 -476.77803435807124])

intercept = -27.40784303
trait_map = None

# Initialize trait map
trait_map = None
first_band = True

# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")

AttributeError: 'NoneType' object has no attribute 'SetGeoTransform'

# d15n

In [48]:
output_tif = r'D:\wenqu\chapter1_2\uas_trait_map\site2c_d15n.tif'


# PLSR model parameters
band_names = ['b74_mean', 'b5_mean', 'b57_mean', 'b58_mean', 'b38_std',
             'b60_mean', 'b25_std', 'b95_std', 'b30_std', 'b9_mean', 'b114_std',
             'b91_std', 'b23_std', 'b18_std', 'b111_mean', 'b14_mean',
             'b101_std', 'b7_mean', 'b122_std', 'b59_std', 'b107_mean',
             'b118_mean', 'b122_mean', 'b13_std', 'b10_std', 'b98_std',
             'b3_mean', 'b99_std', 'b15_std', 'b109_std', 'b14_std', 'b9_std',
             'b8_mean', 'b108_mean', 'b113_std', 'b118_std']

plsr_coefficients = np.array([61.95069832038617,
 -443.8175861956897,
 244.61697666292002,
 161.76757240677037,
 890.6348221654105,
 -179.8869698051848,
 -510.1111454297827,
 149.63785994193933,
 -353.6891690306521,
 -537.7722892932843,
 111.57768804560136,
 -24.366765168206275,
 262.9394749098357,
 -726.5451610814567,
 -22.324825795701166,
 -640.9355498468841,
 -41.70931448381327,
 1007.2321141650461,
 77.62091383540057,
 -535.1730005567188,
 64.55454750642573,
 -69.0054026100846,
 -48.2102651040233,
 282.5904906929578,
 -1322.7513820446086,
 16.7844528529304,
 -157.0443501507085,
 -279.05763846899987,
 -584.5370647143357,
 199.05976457311684,
 1960.7291117127768,
 1359.530026185485,
 616.2063977631504,
 30.11636147145316,
 -182.94787076813563,
 -23.993009150288234])

intercept = -4.16656468

# Initialize trait map
trait_map = None
first_band = True

# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")

Successfully created trait map: D:\wenqu\chapter1_2\uas_trait_map\site2c_d15n.tif
Trait map stats - Min: -23.05, Max: 22.20, Mean: 0.40
